In [1]:
import os
import datetime
from datetime import date,timedelta
import pathlib
from pathlib import Path
import pickle
import glob
import re
import cx_Oracle as oracle
import pandas as pd
import numpy as np

### DB書き込み用のクラスを定義

In [2]:
#CX oracleでMESサーバーへ接続（接続情報以外は定型）
class conn_MES_LWR:
    def __init__(self, host= "10.60.28.21", port="1521", service="psh1dbv",
                       scheme="tabuser",username="tabuser",password="tab123"):        
        self.host = host
        self.port = port
        self.service  = service

        self.scheme   = scheme
        self.username = username
        self.password = password
    
    def __enter__(self):
        
        # tns:Oracleが命名したDB接続用インターフェース技術の名前
        
        # インターフェイスオブジェクトの作成
        self.tns  = oracle.makedsn(self.host, self.port, service_name=self.service) if self.host else None
        # 接続を確立
        self.conn = oracle.connect(self.username, self.password, self.tns) if self.tns else None
        # カーソルの取得
        self.curs = self.conn.cursor() if self.conn else None
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        if self.curs is not None: self.curs.close()
        if self.conn is not None: self.conn.close()

### DB書き込み用の関数

In [3]:
def upload_data(insert_sql=None, df=None):
    
    # DBに格納できるようにデータフレームを二次元配列に変換
    rows = [list(x) for x in df.values]
    
    with conn_MES_LWR() as mesdb:
        # executemany()で複数行のデータを一括でインサート
        mesdb.curs.executemany(insert_sql, rows)
        mesdb.conn.commit()

### DBアップロード用のSQLを定義

##### 170m以上用

In [4]:
insert_data_sql = '''
    INSERT INTO "TABUSER"."Electrode26TA" 
    (
        "ProductionDate", 
        "LotNo", 
        "Sides", 
        "S/E", 
        "ActualThickness(WS)", 
        "ActualThickness(CS)", 
        "ActualThickness(DS)", 
        "FoilWeight(DS)", 
        "CoatingWight(DS)", 
        "CoatingWight(WS)", 
        "SampleWeight(DS)",
        "SampleWeight(WS)", 
        "FillingDensity(DS)", 
        "FillingDensity(WS)", 
        "ElectrodeWidth(withTab)(DS)", 
        "ElectrodeWidth(withoutTab)(DS)", 
        "TabWidth1(DS)", 
        "Pitch(DS)", 
        "TabR1(convex)(DS)",
        "TabR2(convex)(DS)", 
        "TabR1(concave)(DS)", 
        "TabR2(concave)(DS)", 
        "CoatingWidth(withoutProtectiveLayer)(Surface)(DS)",
        "CoatingWidth(withProtectiveLayer)(Surface)(DS)", 
        "CoatingWidth(withoutProtectiveLayer)(Back)(DS)",
        "CoatingWidth(withProtectiveLayer)(Back)(DS)", 
        "HorizontalBurr(Slit)(DS)", 
        "VirticalBurr(Slit)(DS)",
        "HorizontalBurr(Tab)(DS)", 
        "VirticalBurr(Tab)(DS)", 
        "HorizontalBurr(ProtectiveLayer)(DS)", 
        "VirticalBurr(ProtectiveLayer)(DS)", 
        "HigeBurr(DS)", 
        "ElectrodeWidth(withTab)(WS)", 
        "ElectrodeWidth(withoutTab)(WS)", 
        "TabWidth1(WS)", 
        "Pitch(WS)", 
        "TabR1(convex)(WS)",
        "TabR2(convex)(WS)", 
        "TabR1(concave)(WS)", 
        "TabR2(concave)(WS)", 
        "CoatingWidth(withoutProtectiveLayer)(Surface)(WS)",
        "CoatingWidth(withProtectiveLayer)(Surface)(WS)",
        "CoatingWidth(withoutProtectiveLayer)(Back)(WS)",
        "CoatingWidth(withProtectiveLayer)(Back)(WS)", 
        "HorizontalBurr(Slit)(WS)", 
        "VirticalBurr(Slit)(WS)",
        "HorizontalBurr(Tab)(WS)", 
        "VirticalBurr(Tab)(WS)",
        "HorizontalBurr(ProtectiveLayer)(WS)", 
        "VirticalBurr(ProtectiveLayer)(WS)", 
        "HigeBurr(WS)",
        "WindSpeed(Upper)(WS)", 
        "WindSpeed(Upper)(DS)", 
        "WindSpeed(Lower)(WS)", 
        "WindSpeed(Lower)(DS)",
        "Twist(DS)", 
        "Twist(WS)", 
        "Twist(AfterRewinding)(DS)", 
        "Twist(AfterRewinding)(WS)" 
    ) 
    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,
            :21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,
            :41,:42,:43,:44,:45,:46,:47,:48,:49,:50,:51,:52,:53,:54,:55,:56,:57,:58,:59,:60)
'''

##### 170m未満用

In [1]:
insert_data_sql_lt170 = '''
    INSERT INTO "TABUSER"."Electrode26TAlt170" 
    (
        "ProductionDate", 
        "LotNo", 
        "Sides", 
        "S/E", 
        "ActualThickness(WS)", 
        "ActualThickness(CS)", 
        "ActualThickness(DS)", 
        "FoilWeight(DS)", 
        "CoatingWight(DS)", 
        "CoatingWight(WS)", 
        "SampleWeight(DS)",
        "SampleWeight(WS)", 
        "FillingDensity(DS)", 
        "FillingDensity(WS)", 
        "ElectrodeWidth(withTab)(DS)", 
        "ElectrodeWidth(withoutTab)(DS)", 
        "TabWidth1(DS)", 
        "Pitch(DS)", 
        "TabR1(convex)(DS)",
        "TabR2(convex)(DS)", 
        "TabR1(concave)(DS)", 
        "TabR2(concave)(DS)", 
        "CoatingWidth(withoutProtectiveLayer)(Surface)(DS)",
        "CoatingWidth(withProtectiveLayer)(Surface)(DS)", 
        "CoatingWidth(withoutProtectiveLayer)(Back)(DS)",
        "CoatingWidth(withProtectiveLayer)(Back)(DS)", 
        "HorizontalBurr(Slit)(DS)", 
        "VirticalBurr(Slit)(DS)",
        "HorizontalBurr(Tab)(DS)", 
        "VirticalBurr(Tab)(DS)", 
        "HorizontalBurr(ProtectiveLayer)(DS)", 
        "VirticalBurr(ProtectiveLayer)(DS)", 
        "HigeBurr(DS)", 
        "ElectrodeWidth(withTab)(WS)", 
        "ElectrodeWidth(withoutTab)(WS)", 
        "TabWidth1(WS)", 
        "Pitch(WS)", 
        "TabR1(convex)(WS)",
        "TabR2(convex)(WS)", 
        "TabR1(concave)(WS)", 
        "TabR2(concave)(WS)", 
        "CoatingWidth(withoutProtectiveLayer)(Surface)(WS)",
        "CoatingWidth(withProtectiveLayer)(Surface)(WS)",
        "CoatingWidth(withoutProtectiveLayer)(Back)(WS)",
        "CoatingWidth(withProtectiveLayer)(Back)(WS)", 
        "HorizontalBurr(Slit)(WS)", 
        "VirticalBurr(Slit)(WS)",
        "HorizontalBurr(Tab)(WS)", 
        "VirticalBurr(Tab)(WS)",
        "HorizontalBurr(ProtectiveLayer)(WS)", 
        "VirticalBurr(ProtectiveLayer)(WS)", 
        "HigeBurr(WS)",
        "WindSpeed(Upper)(WS)", 
        "WindSpeed(Upper)(DS)", 
        "WindSpeed(Lower)(WS)", 
        "WindSpeed(Lower)(DS)",
        "Twist(DS)", 
        "Twist(WS)", 
        "Twist(AfterRewinding)(DS)", 
        "Twist(AfterRewinding)(WS)" 
    ) 
    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,
            :21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,
            :41,:42,:43,:44,:45,:46,:47,:48,:49,:50,:51,:52,:53,:54,:55,:56,:57,:58,:59,:60)
'''

In [5]:
# 今日の日付データを取得(年・月・日)
year = datetime.date.today().year
month = datetime.date.today().month
day = datetime.date.today().day

In [7]:
# 各ファイルの最終行を記録したdictをデシリアライズ(前月分)
with open('row_num_last_month.pickle', 'rb') as f:
    dict_for_last_month = pickle.load(f)  

# 各ファイルの最終行を記録したdictをデシリアライズ(今月分)
with open('row_num_this_month.pickle', 'rb') as f:
    dict_for_this_month = pickle.load(f)  

# 各ファイルの最終行を記録したdictをデシリアライズ(前月分, 170m未満)
with open('row_num_lt170_last_month.pickle', 'rb') as f:
    dict_for_last_month_lt170 = pickle.load(f)  

# 各ファイルの最終行を記録したdictをデシリアライズ(今月分, 170m未満)
with open('row_num_lt170_this_month.pickle', 'rb') as f:
    dict_for_this_month_lt170 = pickle.load(f)  

In [ ]:
# 月替わりにはシリアライズするデータを初期化
if day == 1:
    # 前月分のpickleデータを変更(170m以上)
    with open('row_num_last_month.pickle', 'wb') as f:
        pickle.dump(dict_for_this_month, f)
    dict_for_last_month = dict_for_this_month
    # 前月分のpickleデータを変更(170m未満)
    with open('row_num_lt170_last_month.pickle', 'wb') as f:
        pickle.dump(dict_for_this_month_lt170, f)
    dict_for_last_month_lt170 = dict_for_this_month_lt170
    
    
    # 今月分のpickleデータを初期化(170m以上)
    dict_for_this_month = {"1-1": None, "1-2": None, "1-3-1": None, "1-3-2": None, "2-1": None, "2-2": None}
    # 今月分のpickleデータを初期化(170m以上)
    dict_for_this_month_lt170 = {"1-1": None, "1-2": None, "1-3-1": None, "1-3-2": None, "2-1": None, "2-2": None}

In [9]:
# 検索対象フォルダのパスを設定し、ファイルを取得
cwd = os.getcwd()
line_folder = ["1期", "2期", "3号機"]
target_date_folder = str(year) + "年" + str(month) + "月"

data_files_last_month = []
data_files_this_month = []

# 毎月10日までは前月分のファイルも覗きにいく
if day <= 10:
    
    # 1月度の場合は昨年の12月度のフォルダを見に行く
    if month == 1:
        target_date_folder2 = str(year - 1) + "年" + str(12) + "月"
    # それ以外の場合は前月度のフォルダを見に行く
    else:    
        target_date_folder2 = str(year) + "年" + str(month - 1) + "月"
        
    for each_line in line_folder:
        
        file_path1 = cwd + "\\★20200701_正極圧縮管理図\\" + each_line + "\\" + target_date_folder2 + "\\*" 
        file_path2 = cwd + "\\★20200701_正極圧縮管理図\\" + each_line + "\\" + target_date_folder2 + "\\"

        for f in glob.glob(file_path1):
            file_name = os.path.split(f)[1]
            if re.findall("\d{4}\.\d{2}_", file_name):

                data_files_last_month.append(file_path2 + file_name)
            
            

for each_line in line_folder:
    
    file_path1 = cwd + "\\★20200701_正極圧縮管理図\\" + each_line + "\\" + target_date_folder + "\\*" 
    file_path2 = cwd + "\\★20200701_正極圧縮管理図\\" + each_line + "\\" + target_date_folder + "\\"
    
    for f in glob.glob(file_path1):
        file_name = os.path.split(f)[1]
        if re.findall("\d{4}\.\d{2}_", file_name):

            data_files_this_month.append(file_path2 + file_name)



In [10]:
# DFのヘッダを作成
s = """ProductionDate
dummy
LotNo
Sides
S/E
ActualThickness(WS)
ActualThickness(CS)
ActualThickness(DS)
FoilWeight(DS)
CoatingWight(DS)
CoatingWight(WS)
SampleWeight(DS)
SampleWeight(WS)
FillingDensity(DS)
FillingDensity(WS)
ElectrodeWidth(withTab)(DS)
ElectrodeWidth(withoutTab)(DS)
TabWidth1(DS)
Pitch(DS)
TabR1(convex)(DS)
TabR2(convex)(DS)
TabR1(concave)(DS)
TabR2(concave)(DS)
CoatingWidth(withoutProtectiveLayer)(Surface)(DS)
CoatingWidth(withProtectiveLayer)(Surface)(DS)
CoatingWidth(withoutProtectiveLayer)(Back)(DS)
CoatingWidth(withProtectiveLayer)(Back)(DS)
HorizontalBurr(Slit)(DS)
VirticalBurr(Slit)(DS)
HorizontalBurr(Tab)(DS)
VirticalBurr(Tab)(DS)
HorizontalBurr(ProtectiveLayer)(DS)
VirticalBurr(ProtectiveLayer)(DS)
HigeBurr(DS)
ElectrodeWidth(withTab)(WS)
ElectrodeWidth(withoutTab)(WS)
TabWidth1(WS)
Pitch(WS)
TabR1(convex)(WS)
TabR2(convex)(WS)
TabR1(concave)(WS)
TabR2(concave)(WS)
CoatingWidth(withoutProtectiveLayer)(Surface)(WS)
CoatingWidth(withProtectiveLayer)(Surface)(WS)
CoatingWidth(withoutProtectiveLayer)(Back)(WS)
CoatingWidth(withProtectiveLayer)(Back)(WS)
HorizontalBurr(Slit)(WS)
VirticalBurr(Slit)(WS)
HorizontalBurr(Tab)(WS)
VirticalBurr(Tab)(WS)
HorizontalBurr(ProtectiveLayer)(WS)
VirticalBurr(ProtectiveLayer)(WS)
HigeBurr(WS)
WindSpeed(Upper)(WS)
WindSpeed(Upper)(DS)
WindSpeed(Lower)(WS)
WindSpeed(Lower)(DS)
Twist(DS)
Twist(WS)
Twist(AfterRewinding)(DS)
Twist(AfterRewinding)(WS)"""

# 文字列を列名のリストに変換
df_header = s.split("\n")

In [11]:
# 170m以上のファイルと未満のファイルに振り分け

# 前月分
if data_files_last_month:
    data_files_last_month_gt_170 = []
    data_files_last_month_lt_170 = []

    for file in data_files_last_month:
        if "以上" in file:
            data_files_last_month_gt_170.append(file)
        elif "未満" in file:
            data_files_last_month_lt_170.append(file)
        

# 今月分
data_files_this_month_gt_170 = []
data_files_this_month_lt_170 = []

for file in data_files_this_month:
    if "以上" in file:
        data_files_this_month_gt_170.append(file)
    elif "未満" in file:
        data_files_this_month_lt_170.append(file)

In [12]:
# シート名のリストを作成
sheet_name_1 = ["1-1", "1-2"]
sheet_name_2 = ["2-1", "2-2"]
sheet_name_3 = ["1-3"]

# 170万メートル以上のデータ処理

### 各ファイルのデータをDFに読み込む (前月分)

In [13]:
df_list = []
# 各ファイルのシート毎にDFを作成

# '3号機'ファイル用にインデックスを準備
idx_for_1_3 = 1
for file in data_files_last_month_gt_170:
    
    if "1期正極" in file:
        
        # 行番号をシリアライズするためにenumerate()のインデックスを利用
        for idx, sheet_name in enumerate(sheet_name_1, start=1):
            df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "1-" + str(idx)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_last_month[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_last_month[key_name] = latest_last_row
                # 前回更新時の最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
    elif "2期正極" in file:
        # 行番号をシリアライズするためにenumerate()のインデックスを利用
        for idx, sheet_name in enumerate(sheet_name_2, start=1):
            df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "2-" + str(idx)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_last_month[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_last_month[key_name] = latest_last_row
                # 前回更新時最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
    elif "3号機" in file:
        df = pd.read_excel(file, sheet_name='1-3', names=df_header, header=12, usecols=range(0, 61))
        # NaN -> 0
        df.fillna(0, inplace=True)
        # dummy列の削除
        df.drop(['dummy'], axis=1, inplace=True)
        # データのある行のみを抽出
        df = df[df['ActualThickness(WS)'] != 0]
        # 生産日のデータを日付に変換
        df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
        key_name = "1-3-" + str(idx_for_1_3)
        # 今回読み込み時の最終行を取得
        latest_last_row = df.shape[0]
        # 前回時の最終行を読み込み
        last_time_last_row = dict_for_last_month[key_name]
        # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
        if latest_last_row != last_time_last_row:
            dict_for_last_month[key_name] = latest_last_row
            # 前回更新時最終行から取得
            df = df.iloc[last_time_last_row:, :]
            df_list.append(df)
        idx_for_1_3 += 1
        

# 各ファイルの最終行を記録したdictをシリアライズ
with open('row_num_last_month.pickle', 'wb') as f:
    pickle.dump(dict_for_last_month, f) 

In [ ]:
# 更新されたDFが1つだけの場合はそのまま取り出す
if len(df_list) == 1:
    df_upload = df_list[0]
elif len(df_list) == 0:
    df_uploat = None
# 複数ある場合はUnion
else:
    df_upload = pd.concat(df_list, ignore_index=True)

In [22]:
# DBにデータをアップロード(前月分・170m以上)
# 更新されたデータがあった場合のみDBにアップロード
if len(df_upload):
    upload_data(insert_data_sql, df_upload)

### 各ファイルのデータをDFに読み込む (今月分)

In [13]:
df_list = []
# 各ファイルのシート毎にDFを作成

# '3号機'ファイル用にインデックスを準備
idx_for_1_3 = 1
for file in data_files_this_month_gt_170:
    
    if "1期正極" in file:
        
        # 行番号をシリアライズするためにenumerate()のインデックスを利用
        for idx, sheet_name in enumerate(sheet_name_1, start=1):
            df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "1-" + str(idx)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_this_month[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_this_month[key_name] = latest_last_row
                # 前回更新時の最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
    elif "2期正極" in file:
        # 行番号をシリアライズするためにenumerate()のインデックスを利用
        for idx, sheet_name in enumerate(sheet_name_2, start=1):
            df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "2-" + str(idx)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_this_month[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_this_month[key_name] = latest_last_row
                # 前回更新時最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
    elif "3号機" in file:
        df = pd.read_excel(file, sheet_name='1-3', names=df_header, header=12, usecols=range(0, 61))
        # NaN -> 0
        df.fillna(0, inplace=True)
        # dummy列の削除
        df.drop(['dummy'], axis=1, inplace=True)
        # データのある行のみを抽出
        df = df[df['ActualThickness(WS)'] != 0]
        # 生産日のデータを日付に変換
        df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
        key_name = "1-3-" + str(idx_for_1_3)
        # 今回読み込み時の最終行を取得
        latest_last_row = df.shape[0]
        # 前回時の最終行を読み込み
        last_time_last_row = dict_for_this_month[key_name]
        # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
        if latest_last_row != last_time_last_row:
            dict_for_this_month[key_name] = latest_last_row
            # 前回更新時最終行から取得
            df = df.iloc[last_time_last_row:, :]
            df_list.append(df)
        idx_for_1_3 += 1


# 各ファイルの最終行を記録したdictをシリアライズ
with open('row_num_this_month.pickle', 'wb') as f:
    pickle.dump(dict_for_this_month, f) 

In [ ]:
# 更新されたDFが1つだけの場合はそのまま取り出す
if len(df_list) == 1:
    df_upload = df_list[0]
elif len(df_list) == 0:
    df_uploat = None
# 複数ある場合はUnion
else:
    df_upload = pd.concat(df_list, ignore_index=True)

In [22]:
# DBにデータをアップロード(今月分・170m以上)
# 更新されたデータがあった場合のみDBにアップロード
if len(df_upload):
    upload_data(insert_data_sql, df_upload)

# 170万メートル未満のデータ処理

### 各ファイルのデータをDFに読み込む (前月分)

In [13]:
df_list = []
# 各ファイルのシート毎にDFを作成

# '3号機'ファイル用にインデックスを準備
idx_for_1_3 = 1

if data_file_last_month_lt_170:
    for file in data_files_last_month_lt_170:

        if "1期正極" in file:

            # 行番号をシリアライズするためにenumerate()のインデックスを利用
            for idx, sheet_name in enumerate(sheet_name_1, start=1):
                df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
                # NaN -> 0
                df.fillna(0, inplace=True)
                # dummy列の削除
                df.drop(['dummy'], axis=1, inplace=True)
                # データのある行のみを抽出
                df = df[df['ActualThickness(WS)'] != 0]
                # 生産日のデータを日付に変換
                df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
                key_name = "1-" + str(idx)
                # 今回読み込み時の最終行を取得
                latest_last_row = df.shape[0]
                # 前回時の最終行を読み込み
                last_time_last_row = dict_for_last_month_lt170[key_name]
                # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
                if latest_last_row != last_time_last_row:
                    dict_for_last_month_lt170[key_name] = latest_last_row
                    # 前回更新時の最終行から取得
                    df = df.iloc[last_time_last_row:, :]
                    df_list.append(df)
        elif "2期正極" in file:
            # 行番号をシリアライズするためにenumerate()のインデックスを利用
            for idx, sheet_name in enumerate(sheet_name_2, start=1):
                df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
                # NaN -> 0
                df.fillna(0, inplace=True)
                # dummy列の削除
                df.drop(['dummy'], axis=1, inplace=True)
                # データのある行のみを抽出
                df = df[df['ActualThickness(WS)'] != 0]
                # 生産日のデータを日付に変換
                df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
                key_name = "2-" + str(idx)
                # 今回読み込み時の最終行を取得
                latest_last_row = df.shape[0]
                # 前回時の最終行を読み込み
                last_time_last_row = dict_for_last_month_lt170[key_name]
                # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
                if latest_last_row != last_time_last_row:
                    dict_for_last_month_lt170[key_name] = latest_last_row
                    # 前回更新時最終行から取得
                    df = df.iloc[last_time_last_row:, :]
                    df_list.append(df)
        elif "3号機" in file:
            df = pd.read_excel(file, sheet_name='1-3', names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "1-3-" + str(idx_for_1_3)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_last_month_lt170[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_last_month_lt170[key_name] = latest_last_row
                # 前回更新時最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
            idx_for_1_3 += 1
        

# 各ファイルの最終行を記録したdictをシリアライズ
with open('row_num_last_month.pickle', 'wb') as f:
    pickle.dump(dict_for_last_month_lt170, f) 

In [ ]:
# 更新されたDFが1つだけの場合はそのまま取り出す
if len(df_list) == 1:
    df_upload = df_list[0]
elif len(df_list) == 0:
    df_upload = None
# 複数ある場合はUnion
else:
    df_upload = pd.concat(df_list, ignore_index=True)

In [22]:
# DBにデータをアップロード
# 更新されたデータがあった場合のみDBにアップロード
if len(df_upload):
    upload_data(insert_data_sql_lt170, df_upload)

### 各ファイルのデータをDFに読み込む (今月分)

In [13]:
df_list = []
# 各ファイルのシート毎にDFを作成

# '3号機'ファイル用にインデックスを準備
idx_for_1_3 = 1

if data_files_this_month_lt_170:
    for file in data_files_this_month_lt_170:

        if "1期正極" in file:

            # 行番号をシリアライズするためにenumerate()のインデックスを利用
            for idx, sheet_name in enumerate(sheet_name_1, start=1):
                df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
                # NaN -> 0
                df.fillna(0, inplace=True)
                # dummy列の削除
                df.drop(['dummy'], axis=1, inplace=True)
                # データのある行のみを抽出
                df = df[df['ActualThickness(WS)'] != 0]
                # 生産日のデータを日付に変換
                df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
                key_name = "1-" + str(idx)
                # 今回読み込み時の最終行を取得
                latest_last_row = df.shape[0]
                # 前回時の最終行を読み込み
                last_time_last_row = dict_for_this_month_lt170[key_name]
                # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
                if latest_last_row != last_time_last_row:
                    dict_for_this_month_lt170[key_name] = latest_last_row
                    # 前回更新時の最終行から取得
                    df = df.iloc[last_time_last_row:, :]
                    df_list.append(df)
        elif "2期正極" in file:
            # 行番号をシリアライズするためにenumerate()のインデックスを利用
            for idx, sheet_name in enumerate(sheet_name_2, start=1):
                df = pd.read_excel(file, sheet_name=sheet_name, names=df_header, header=12, usecols=range(0, 61))
                # NaN -> 0
                df.fillna(0, inplace=True)
                # dummy列の削除
                df.drop(['dummy'], axis=1, inplace=True)
                # データのある行のみを抽出
                df = df[df['ActualThickness(WS)'] != 0]
                # 生産日のデータを日付に変換
                df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
                key_name = "2-" + str(idx)
                # 今回読み込み時の最終行を取得
                latest_last_row = df.shape[0]
                # 前回時の最終行を読み込み
                last_time_last_row = dict_for_this_month_lt170[key_name]
                # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
                if latest_last_row != last_time_last_row:
                    dict_for_this_month_lt170[key_name] = latest_last_row
                    # 前回更新時最終行から取得
                    df = df.iloc[last_time_last_row:, :]
                    df_list.append(df)
        elif "3号機" in file:
            df = pd.read_excel(file, sheet_name='1-3', names=df_header, header=12, usecols=range(0, 61))
            # NaN -> 0
            df.fillna(0, inplace=True)
            # dummy列の削除
            df.drop(['dummy'], axis=1, inplace=True)
            # データのある行のみを抽出
            df = df[df['ActualThickness(WS)'] != 0]
            # 生産日のデータを日付に変換
            df["ProductionDate"] = df["ProductionDate"].apply(get_datetime)
            key_name = "1-3-" + str(idx_for_1_3)
            # 今回読み込み時の最終行を取得
            latest_last_row = df.shape[0]
            # 前回時の最終行を読み込み
            last_time_last_row = dict_for_this_month_lt170[key_name]
            # 新たにデータが追加されていた場合のみdictの書き換えとDFの読み込みを行う
            if latest_last_row != last_time_last_row:
                dict_for_this_month_lt170[key_name] = latest_last_row
                # 前回更新時最終行から取得
                df = df.iloc[last_time_last_row:, :]
                df_list.append(df)
            idx_for_1_3 += 1


# 各ファイルの最終行を記録したdictをシリアライズ
with open('row_num_this_month.pickle', 'wb') as f:
    pickle.dump(dict_for_this_month_lt170, f) 

In [ ]:
# 更新されたDFが1つだけの場合はそのまま取り出す
if len(df_list) == 1:
    df_upload = df_list[0]
elif len(df_list) == 0:
    df_upload = 0
# 複数ある場合はUnion
else:
    df_upload = pd.concat(df_list, ignore_index=True)

In [22]:
# DBにデータをアップロード
# 更新されたデータがあった場合のみDBにアップロード
if len(df_upload):
    upload_data(insert_data_sql_lt170, df_upload)